In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [73]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the U-Net generator
class GeneratorUNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GeneratorUNet, self).__init__()

        # Encoder (downsampling)
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
            # nn.Conv2d(512, 1024, kernel_size=4, stride=2, padding=1),
            # nn.BatchNorm2d(1024),
            # nn.ReLU()
        )

        # Decoder (upsampling)
        self.decoder = nn.Sequential(
            # nn.ConvTranspose2d(1024, 512, kernel_size=4, stride=2, padding=1),
            # nn.BatchNorm2d(512),
            # nn.ReLU(),
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, out_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh()  # Output normalized between -1 and 1
        )

    def forward(self, x):
        # print(x.shape)
        x_enc = self.encoder(x)
        x_dec = self.decoder(x_enc)
        # print(x.shape)
        return x_dec

# Example usage
latent_size = 100
img_channels = 3  # Adjust based on your dataset (e.g., 3 for RGB images)
img_size = 64  # Adjust based on your desired image size
generator = GeneratorUNet(1, 1)

# Generate random input noise
z = torch.randn(128, 1, 28, 28)  # Batch size of 1
fake_images = generator(z)
# print(fake_images.shape)  # Check the output shape


In [74]:
import torch
import torch.nn as nn

class Discriminator(nn.Module):
    def __init__(self, img_channels):
        super(Discriminator, self).__init__()
        self.img_channels = img_channels
        # self.img_size = img_size
        
        self.model = nn.Sequential(
            nn.Conv2d(img_channels, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),
            nn.Flatten(),
            nn.Linear(512, 1),  # Adjust output size based on your needs
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.model(x)
        return x


In [75]:
disk = Discriminator(1)
disk(fake_images)

tensor([[0.5097],
        [0.6841],
        [0.5130],
        [0.3640],
        [0.4628],
        [0.5039],
        [0.5317],
        [0.4412],
        [0.5102],
        [0.5046],
        [0.5192],
        [0.5257],
        [0.5866],
        [0.5123],
        [0.4001],
        [0.5463],
        [0.4396],
        [0.5019],
        [0.4355],
        [0.4701],
        [0.6067],
        [0.4346],
        [0.4840],
        [0.4237],
        [0.4731],
        [0.5159],
        [0.4757],
        [0.6613],
        [0.5537],
        [0.3980],
        [0.4930],
        [0.4292],
        [0.4965],
        [0.4970],
        [0.5077],
        [0.2646],
        [0.4360],
        [0.5120],
        [0.5929],
        [0.4995],
        [0.5884],
        [0.4188],
        [0.5534],
        [0.6215],
        [0.4942],
        [0.4972],
        [0.4525],
        [0.5334],
        [0.3356],
        [0.4937],
        [0.6244],
        [0.5359],
        [0.4435],
        [0.5611],
        [0.5799],
        [0

In [78]:
# Define the training function
import torchvision.utils as vutils
def train(discriminator, generator, dataloader, num_epochs=50, batch_size=128, latent_size=100, lr=0.0002):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    discriminator.to(device)
    generator.to(device)
    
    criterion = nn.BCELoss()
    d_optimizer = optim.Adam(discriminator.parameters(), lr=lr)
    g_optimizer = optim.Adam(generator.parameters(), lr=lr)
    
    fixed_noise = torch.randn(batch_size, 1, 28, 28, device=device)  # Fixed noise for tracking generator progress
    
    for epoch in range(num_epochs):
        for i, real_images in enumerate(dataloader):

            real_images = real_images[0].to(device)
            batch_size = real_images.size(0)

            real_labels = torch.ones(batch_size, 1, device=device)
            fake_labels = torch.zeros(batch_size, 1, device=device)
            
            # Train discriminator with real images
            outputs = discriminator(real_images)
            d_loss_real = criterion(outputs, real_labels)
            real_score = outputs.mean().item()
            
            # Train discriminator with fake images
            z = torch.randn(real_images.shape, device=device)
            fake_images = generator(z)
            outputs = discriminator(fake_images.detach())
            d_loss_fake = criterion(outputs, fake_labels)
            fake_score = outputs.mean().item()
            
            d_loss = d_loss_real + d_loss_fake
            discriminator.zero_grad()
            d_loss.backward()
            d_optimizer.step()
            
            # Train generator
            z = torch.randn(real_images.shape, device=device)
            fake_images = generator(z)
            outputs = discriminator(fake_images)
            g_loss = criterion(outputs, real_labels)
            
            generator.zero_grad()
            g_loss.backward()
            g_optimizer.step()
            
            if (i+1) % 200 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], '
                      f'D_loss: {d_loss.item():.4f}, G_loss: {g_loss.item():.4f}, '
                      f'Real Score: {real_score:.4f}, Fake Score: {fake_score:.4f}')
                
        # Save generated images for tracking progress
        with torch.no_grad():
            fake_samples = generator(fixed_noise).detach()
            fake_grid = vutils.make_grid(fake_samples, nrow=8, normalize=True)
            vutils.save_image(fake_grid, f'generated_images_epoch_{epoch+1}.png', normalize=True)


# Set random seed for reproducibility
torch.manual_seed(42)

# Define dataset and dataloader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)

# Initialize discriminator and generator
img_channels = 1  # Grayscale images for MNIST
img_size = 28  # Image size for MNIST
latent_size = 100

# discriminator = Discriminator(img_channels)
# generator = GeneratorUNet(img_channels, img_channels)

# Train the GAN
train(discriminator, generator, train_loader, num_epochs=1000, batch_size=128, latent_size=latent_size, lr=0.0002)


Epoch [1/1000], Step [200/469], D_loss: 0.0233, G_loss: 4.3621, Real Score: 0.9903, Fake Score: 0.0134
Epoch [1/1000], Step [400/469], D_loss: 0.1407, G_loss: 5.1711, Real Score: 0.9056, Fake Score: 0.0055
Epoch [2/1000], Step [200/469], D_loss: 0.0084, G_loss: 5.4388, Real Score: 0.9963, Fake Score: 0.0046


KeyboardInterrupt: 

In [79]:
# Train the GAN
train(discriminator, generator, train_loader, num_epochs=50, batch_size=512, latent_size=100, lr=0.0002)

Epoch [1/50], Step [200/469], D_loss: 0.0197, G_loss: 4.6285, Real Score: 0.9920, Fake Score: 0.0114
Epoch [1/50], Step [400/469], D_loss: 0.0048, G_loss: 5.5312, Real Score: 0.9988, Fake Score: 0.0035
Epoch [2/50], Step [200/469], D_loss: 0.0060, G_loss: 5.8955, Real Score: 0.9971, Fake Score: 0.0031
Epoch [2/50], Step [400/469], D_loss: 0.0080, G_loss: 6.3209, Real Score: 0.9946, Fake Score: 0.0026
Epoch [3/50], Step [200/469], D_loss: 0.0240, G_loss: 5.8034, Real Score: 0.9805, Fake Score: 0.0029
Epoch [3/50], Step [400/469], D_loss: 0.0037, G_loss: 5.9918, Real Score: 0.9991, Fake Score: 0.0028
Epoch [4/50], Step [200/469], D_loss: 0.0037, G_loss: 6.2775, Real Score: 0.9982, Fake Score: 0.0019
Epoch [4/50], Step [400/469], D_loss: 0.0137, G_loss: 6.0207, Real Score: 0.9977, Fake Score: 0.0112
Epoch [5/50], Step [200/469], D_loss: 0.0148, G_loss: 5.7579, Real Score: 0.9908, Fake Score: 0.0047
Epoch [5/50], Step [400/469], D_loss: 0.0071, G_loss: 5.9438, Real Score: 0.9976, Fake Scor